<a href="https://colab.research.google.com/github/JaquelineEspinoza/Investigaci-n-de-operaciones/blob/main/Inventario%20con%20descuento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inventario con descuento**

In [31]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np

# **Ejemplo**

LubeCar se especializa en cambios rápidos de aceite para motor de automóvil. El servicio compra aceite para motor a granel, a \$3.00 por galón. Si LubeCar compra más de 100 galones, obtiene un descuento de \$2.50 por galón. En el servicio se atienden unos 150 autos diarios, y cada cambio de aceite requiere de 1.25 galones. LubeCar guarda el aceite a granel con un costo de \$0.02 por galón y por día. También, el costo de colocar un pedido de aceite a granel es de \$20. Hay un tiempo de 2 días para la entrega. Determine la política óptima de inventario

In [50]:
# Definimos los simbolos y funciones
c1, c2, d, h, K, L, Q, Qop, t1=sp.symbols("c1, c2, d, h, K, L, Q, Qop, t1")
T=sp.Function("T")
Qo=sp.Function("Qo")

In [53]:
c1=3
c2=2.5
d=187.5
h=0.02
K=20
L=2
q=1000
#Qop=612.3724

Paso 1. Calcular Q óptimo

In [54]:
def Qop(K,d,h):
    return np.sqrt((2*K*d)/h)

In [55]:
Qo=Qop(K,d,h)

In [20]:
Qo

Qo

Paso 2. Determinar la zona en que se encuentra q

In [56]:
def T(Q):
    if Q <= q:
        f=c1*d+K*d/Q+h*Q/2
    else:
        f=c2*d+K*d/Q+h*Q/2
    return f

In [57]:
eq=sp.Eq(Q**2 + (2/h)*(c2*d-T(Qo))*Q+2*K*d/h,0)

In [59]:
eq

Eq(Q**2 - 10599.7448713916*Q + 375000.0, 0)

In [74]:
sol=sp.solve(eq,Q)

In [75]:
sol

[(-c2*d - sqrt(-2.0*K*d*h + c2**2*d**2 - 2.0*c2*d*T(122474487139159/200000000000) + T(122474487139159/200000000000)**2) + T(122474487139159/200000000000))/h,
 (-c2*d + sqrt(-2.0*K*d*h + c2**2*d**2 - 2.0*c2*d*T(122474487139159/200000000000) + T(122474487139159/200000000000)**2) + T(122474487139159/200000000000))/h]

In [76]:
s=sol[1] #Aquí debemos seleccionar la solución que sea mayor que q

In [77]:
s

(-c2*d + sqrt(-2.0*K*d*h + c2**2*d**2 - 2.0*c2*d*T(122474487139159/200000000000) + T(122474487139159/200000000000)**2) + T(122474487139159/200000000000))/h

**Ejercicio 1:** Escriba una función que calcula la Q óptima. Es decir, que resuelva de manera interna la ecuación cuadrática y seleccione la solución correcta e indique en qué región está.

In [132]:
def calcular_Q_optima(c1, c2, h, K, d, q):
    """
    Calcula la cantidad óptima de pedido considerando descuento por cantidad

    Parámetros:
    c1: costo por unidad sin descuento
    c2: costo por unidad con descuento (cuando Q > q)
    h: costo de mantenimiento por unidad y por día
    K: costo de colocar un pedido
    d: demanda diaria
    q: cantidad mínima para obtener descuento
    """

    import numpy as np
    import sympy as sp
    # Paso 1: Calcular Qop
    Qop = np.sqrt((2 * K * d) / h)

    # Paso 2: Calcular T(Qop) usando c1
    T_Qop = c1 * d + K * d / Qop + h * Qop / 2

    # Paso 3: Resolver la ecuación cuadrática para Q (para c2)
    Q = sp.symbols('Q')
    ecuacion = sp.Eq(Q**2 + (2/h)*(c2*d - T_Qop)*Q + 2*K*d/h, 0)
    soluciones = sp.solve(ecuacion, Q)

    # Seleccionar la solución mayor que q
    Q_s = max([float(sol) for sol in soluciones if float(sol) > q])

    # Paso 4: Determinar en qué zona estamos
    if Qop <= q:
        # Zona I: Qop < q < Q_s (si existe Q_s)
        Q_optima = Qop
        region = "I"
    elif q < Qop < Q_s:
        # Zona II: Qop está entre q y Q_s
        Q_optima = q
        region = "II"
    else:
        # Zona III: Q_s es la óptima
        Q_optima = Q_s
        region = "III"

    return Q_optima, region, Qop, Q_s
    c1=3
    c2=0.5
    h=0.02
    K=20
    d=150
    q=100

**Ejercicio 2:** Escribir una función que reciba los datos del problema y devuelva la política óptima del inventario.

In [80]:
def politica_optima_inventario(c1, c2, d, h, K, q, L):
    """
    Determina la política óptima de inventario con descuento por cantidad

    Parámetros:
    c1: costo por unidad sin descuento
    c2: costo por unidad con descuento (cuando Q > q)
    d: demanda diaria
    h: costo de mantenimiento por unidad y por día
    K: costo de colocar un pedido
    q: cantidad mínima para obtener descuento
    L: tiempo de entrega en días
    """
    import numpy as np

    # Calcular Q óptima
    Q_optima, region, Qop, Q_s = calcular_Q_optima(c1, c2, h, K, d, q)

    # Calcular punto de reorden
    t1 = Q_optima / d  # ciclo del inventario en días
    n = int(np.floor(L / t1))  # número de ciclos completos durante el lead time

    # Tiempo de retraso efectivo
    L_eff = L - n * t1

    # Punto de reorden (inventario al cual hacer el pedido)
    punto_reorden = L_eff * d

    # Política óptima
    politica = {
        'Cantidad_pedido': Q_optima,
        'Punto_reorden': punto_reorden,
        'Region': region,
        'Qop': Qop,
        'Qs': Q_s,
        'Ciclo_inventario_dias': t1,
        'Num_ciclos_lead_time': n,
        'Lead_time_efectivo_dias': L_eff
    }

    return politica

# Ejemplo de uso con los datos del problema
c1 = 3
c2 = 2.5
d = 187.5  # 150 autos × 1.25 galones
h = 0.02
K = 20
q = 1000
L = 2

# Calcular política óptima
politica = politica_optima_inventario(c1, c2, d, h, K, q, L)

print("=== POLÍTICA ÓPTIMA DE INVENTARIO ===\n")
print(f"1. Cantidad óptima de pedido: {politica['Cantidad_pedido']:.2f} galones")
print(f"2. Punto de reorden: {politica['Punto_reorden']:.2f} galones")
print(f"3. Zona: {politica['Region']}")
print(f"4. Qop (EOQ estándar): {politica['Qop']:.2f}")
print(f"5. Qs (solución cuadrática): {politica['Qs']:.2f}")
print(f"6. Ciclo de inventario: {politica['Ciclo_inventario_dias']:.2f} días")
print(f"7. Número de ciclos durante lead time: {politica['Num_ciclos_lead_time']}")
print(f"8. Lead time efectivo: {politica['Lead_time_efectivo_dias']:.2f} días")

print("\nResumen de la política:")
if politica['Region'] == "I":
    print(f"• Pedir {politica['Cantidad_pedido']:.0f} galones cuando el inventario caiga a {politica['Punto_reorden']:.0f} galones (sin descuento)")
elif politica['Region'] == "II":
    print(f"• Pedir {politica['Cantidad_pedido']:.0f} galones cuando el inventario caiga a {politica['Punto_reorden']:.0f} galones (con descuento, zona intermedia)")
else:  # Region III
    print(f"• Pedir {politica['Cantidad_pedido']:.0f} galones cuando el inventario caiga a {politica['Punto_reorden']:.0f} galones (con descuento)")

=== POLÍTICA ÓPTIMA DE INVENTARIO ===

1. Cantidad óptima de pedido: 612.37 galones
2. Punto de reorden: 375.00 galones
3. Zona: I
4. Qop (EOQ estándar): 612.37
5. Qs (solución cuadrática): 10564.25
6. Ciclo de inventario: 3.27 días
7. Número de ciclos durante lead time: 0
8. Lead time efectivo: 2.00 días

Resumen de la política:
• Pedir 612 galones cuando el inventario caiga a 375 galones (sin descuento)
